In [3]:
from langchain.chat_models import init_chat_model

llm = init_chat_model("openai:gpt-4o-mini")

In [6]:
from langchain_community.utilities import SQLDatabase
from dotenv import load_dotenv
import os

# Cargar variables desde .env
load_dotenv()

# Leer valores
host = os.getenv("POSTGRES_HOST")
port = os.getenv("POSTGRES_PORT")
db_name = os.getenv("POSTGRES_DB")
user = os.getenv("POSTGRES_USER")
password = os.getenv("POSTGRES_PASSWORD")

# Construir URI de conexión PostgreSQL
uri = f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{db_name}"

# Crear conexión
db = SQLDatabase.from_uri(uri)

# Validar conexión
print(f"Dialect: {db.dialect}")
print(f"Available tables: {db.get_usable_table_names()}")
print(f'Sample output: {db.run("SELECT * FROM transparencia LIMIT 5;")}')

Dialect: postgresql
Available tables: ['contratos', 'orden_servicio', 'sunat', 'transparencia']
Sample output: [('20131370645', 'MINISTERIO DE ECONOMIA Y FINANZAS', '19,312,898,790.58', 1), ('20414671773', 'FONDO MIVIVIENDA S.A.', '12,293,205,840.88', 1), ('20100128218', 'PETROLEOS DEL PERU PETROPERU SA', '9,905,757,838.69', 1), ('20557410008', 'METRO DE LIMA LINEA 2 S.A.', '9,649,129,064.13', 1), ('20100047218', 'BANCO DE CREDITO DEL PERU', '8,015,071,213.50', 1)]


In [7]:
from langchain_community.agent_toolkits import SQLDatabaseToolkit

toolkit = SQLDatabaseToolkit(db=db, llm=llm)

tools = toolkit.get_tools()

for tool in tools:
    print(f"{tool.name}: {tool.description}\n")

sql_db_query: Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.

sql_db_schema: Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3

sql_db_list_tables: Input is an empty string, output is a comma-separated list of tables in the database.

sql_db_query_checker: Use this tool to double check if your query is correct before executing it. Always use this tool before executing a query with sql_db_query!



In [10]:
from langgraph.prebuilt import create_react_agent

system_prompt = """
You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct {dialect} query to run,
then look at the results of the query and return the answer. Unless the user
specifies a specific number of examples they wish to obtain, always limit your
query to at most {top_k} results.

You can order the results by a relevant column to return the most interesting
examples in the database. Never query for all the columns from a specific table,
only ask for the relevant columns given the question.

You MUST double check your query before executing it. If you get an error while
executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the
database.

To start you should ALWAYS look at the tables in the database to see what you
can query. Do NOT skip this step.

Then you should query the schema of the most relevant tables.
""".format(
    dialect=db.dialect,
    top_k=10,
)

agent = create_react_agent(
    llm,
    tools,
    prompt=system_prompt,
)

In [11]:
question = "Cual es el presupuesto de empresas q fueron creados en el 2022?"

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Cual es el presupuesto de empresas q fueron creados en el 2022?
================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (call_isEaQgZTQfeBLaOyRadzqqmP)
 Call ID: call_isEaQgZTQfeBLaOyRadzqqmP
  Args:
================================= Tool Message =================================
Name: sql_db_list_tables

contratos, orden_servicio, sunat, transparencia
================================== Ai Message ==================================
Tool Calls:
  sql_db_schema (call_0HUfekfmbs47BMjq5dhsSNyz)
 Call ID: call_0HUfekfmbs47BMjq5dhsSNyz
  Args:
    table_names: contratos
  sql_db_schema (call_PlCg3rF78rbmC1qiF6yyTYHD)
 Call ID: call_PlCg3rF78rbmC1qiF6yyTYHD
  Args:
    table_names: orden_servicio
  sql_db_schema (call_ob5qANsd4Mel3VJKdjPT3TZK)
 Call ID: call_ob5qANsd4Mel3VJKdjPT3TZK
  Args:
    table_names: sunat
  sql_db_schema (call_gycNSUy966hrtNhJQmNf

In [12]:
question = "Lista las empresas que fueron creadas en algun més del 2022 y en menos de 4 meses de su creación tuvo contratos u ordenes de servicios de más de 50000 soles?"

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Lista las empresas que fueron creadas en algun més del 2022 y en menos de 4 meses de su creación tuvo contratos u ordenes de servicios de más de 50000 soles?
================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (call_ufeTs0Xwv5BkxJ7WifKPkGri)
 Call ID: call_ufeTs0Xwv5BkxJ7WifKPkGri
  Args:
================================= Tool Message =================================
Name: sql_db_list_tables

contratos, orden_servicio, sunat, transparencia
================================== Ai Message ==================================
Tool Calls:
  sql_db_schema (call_Ctc23FxNMdxe2AUhsWLNkIpn)
 Call ID: call_Ctc23FxNMdxe2AUhsWLNkIpn
  Args:
    table_names: contratos
  sql_db_schema (call_XG4OeEB48RJYbvkxnshCgCGj)
 Call ID: call_XG4OeEB48RJYbvkxnshCgCGj
  Args:
    table_names: orden_servicio
  sql_db_schema (call_waiAAstYll0MOoUbvQn1BwHE)
 Call ID: call_waiA